In [100]:
import torch
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn.functional as F

In [1]:
words = open('names.txt', 'r').read().splitlines()

In [42]:
N = torch.zeros((27,27), dtype=torch.int32)

In [43]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

In [45]:
for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1, ix2 = stoi[ch1], stoi[ch2]
        N[ix1, ix2] += 1

In [49]:
p = N[0].float()
p = p / p.sum()

g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()

itos[ix]

'j'

In [94]:
P = (N+1).float()
P /= P.sum(1, keepdim=True)

In [95]:
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    out = []
    ix = 0
    while True:
        p = P[ix]
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])
        if ix == 0:
            break
    
    print(''.join(out))

junide.
janasah.
p.
cony.
a.


In [159]:
log_likelihood = 0.0
n = 0

for w in words[:4]:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1, ix2 = stoi[ch1], stoi[ch2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        n += 1
        # print(f'{ch1}{ch2}: {prob:.4f}: {logprob:.4f}')
print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n=}')

log_likelihood=tensor(-60.3371)
nll=tensor(60.3371)
nll/n=tensor(2.4135)


In [173]:
xs, ys = [], []

for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1, ix2 = stoi[ch1], stoi[ch2]
        xs.append(ix1)
        ys.append(ix2)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print(f'num examples: {num}')

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 27), generator=g, requires_grad=True)

num examples: 228146


In [175]:
for k in range(100):
    
    xenc = F.one_hot(xs, num_classes=27).float()
    logits = xenc @ W
    counts = logits.exp()
    prob = counts / counts.sum(1, keepdims=True)
    loss = -prob[torch.arange(num), ys].log().mean()
    print(loss.item())
    
    W.grad = None
    loss.backward()
    W.data += -50 * W.grad

2.4726526737213135
2.4724342823028564
2.4722204208374023
2.472010850906372
2.471806049346924
2.4716055393218994
2.4714088439941406
2.4712166786193848
2.4710280895233154
2.470843553543091
2.4706623554229736
2.4704854488372803
2.4703118801116943
2.4701414108276367
2.4699742794036865
2.4698104858398438
2.4696500301361084
2.469492197036743
2.4693379402160645
2.4691860675811768
2.4690370559692383
2.468890905380249
2.468747854232788
2.46860671043396
2.46846866607666
2.468332529067993
2.4681990146636963
2.4680678844451904
2.4679391384124756
2.4678127765655518
2.46768856048584
2.4675662517547607
2.4674463272094727
2.4673283100128174
2.467211961746216
2.4670979976654053
2.4669854640960693
2.4668753147125244
2.466766834259033
2.4666597843170166
2.466554641723633
2.466451406478882
2.4663493633270264
2.4662487506866455
2.4661502838134766
2.4660532474517822
2.4659576416015625
2.4658634662628174
2.465770721435547
2.465679407119751
2.4655895233154297
2.465500593185425
2.4654133319854736
2.46532750129